In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys

MAIN_MODULE_PATH = os.path.join(os.getcwd(), '..', '..', '..')
sys.path.append(MAIN_MODULE_PATH)

In [ ]:
from collections import Counter, defaultdict

import numpy as np
import scipy
import pandas as pd
import seaborn as sns
from sklearn.metrics import ndcg_score

from src.dataset.negatives import sample_negatives
from src.defaults import REPO_DS, TRAIN_DS, TEST_DS
from src.metrics.surprise_metrics import precision_recall_at_k, ndcg_at_k

In [ ]:
def prepare_data():
#     repository_features = pd.read_csv(REPO_DS)
    train_interactions = pd.read_csv(TRAIN_DS)
    test_interactions = pd.read_csv(TEST_DS)
    
    train_negatives = sample_negatives(train_interactions, len(train_interactions), seed=1293)
    test_negatives = sample_negatives(test_interactions, len(test_interactions), seed=1293)
    
    train_interactions['rating'] = 1
    train_negatives['rating'] = 0
    
    test_interactions['rating'] = 1
    test_negatives['rating'] = 0

    train = pd.concat([train_interactions, train_negatives])
    test = pd.concat([test_interactions, test_negatives])
    
    return train, test

In [ ]:
train, test = prepare_data()

# KNNsurprise

In [ ]:
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate, KFold, train_test_split

data = Dataset.load_from_df(train, Reader(rating_scale=(0, 1)))

trainset = data.build_full_trainset()

results = []


for model in [KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline]:
    for sim_measure_name in ['cosine', 'msd', 'pearson']:
        algo = model(sim_options = {'name': sim_measure_name})
        algo.fit(trainset)
        predictions = [algo.predict(row.full_name, row.repo_requirements, row.rating) for row in test.itertuples()]
        precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=0.5)
        ndcg = ndcg_at_k(predictions, k=10, threshold=0.5)
        precision = sum(prec for prec in precisions.values()) / len(precisions)
        recall = sum(rec for rec in recalls.values()) / len(recalls)

        results.append({'model': model.__name__, 'similarity_measure': sim_measure_name, 'precision': precision, 'recall': recall, 'ndcg': ndcg})

results = pd.DataFrame(results)

In [ ]:
results.sort_values(by='ndcg', ascending=False)